In [1]:
import os
import json
from typing import List, Literal, Optional, Text, Union, Dict, Any
from IPython.display import Image, display
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod
from agents.agents_modules.workflow import build_agent_workflow, workers_dict
from agents.dataloader import load_dataset_by_name, extract_example

/home/chinonso/anaconda3/envs/lang2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch version 2.8.0.dev20250515+cu128 available.


In [2]:
def save_result_to_json(state: dict, filename: str = "result.json", directory: str = "results") -> None:
    """
    Saves the given agent workflow state to a JSON file in a specified directory.

    Args:
        state (dict): The final state returned by the agent graph execution.
        filename (str): The name of the JSON file to save.
        directory (str): The directory where the file will be saved.

    Raises:
        IsADirectoryError: If the target file path is a directory.
    """

    file_path = os.path.join(directory, filename)
    os.makedirs(directory, exist_ok=True)

    if os.path.isdir(file_path):
        raise IsADirectoryError(f"Cannot write to '{file_path}' because it is a directory.")

    # Recursively convert objects to serializable types
    def make_serializable(obj):
        if isinstance(obj, list):
            return [make_serializable(x) for x in obj]
        elif hasattr(obj, "model_dump"):  # Pydantic BaseModel
            return obj.model_dump()
        elif isinstance(obj, dict):
            return {k: make_serializable(v) for k, v in obj.items()}
        else:
            return obj

    serializable_state = make_serializable(dict(state))

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(serializable_state, f, indent=4)

    print(f"Results saved to {file_path}")


In [3]:
name = "totto"  # Change this to the dataset you want to load
num = 345 # Sample id

data = load_dataset_by_name(name)
sample = extract_example(name, data["validation"][num])
data = sample['input']
ground_truth = sample['references']
target = sample['target']

query = f"""You are an agent designed to generate text from data for a data-to-text natural language generation. You can be provided data in the form of xml, table, meaning representations, graphs etc. 
Your task is to generate the appropriate text given the data information without omitting any field or adding extra information in essence called hallucination.
Here is the data generate text using table data:
{data}"""

initial_state = {
    "input": query,
    "raw_input": data,
    "plan": [],
    "result_steps": [],
    "response": "",
    "chat_history": [],
    "next": "",
    "next_input": "",
    "agent_outcome": "",
    "current_step": 0,
    "team_iterations": 0,
    "recursion_limit": 60,
}

print(f"\nInput: {data} \n")
print(f"Target: {target} \n")
print(f"Ground Truth: {ground_truth}")

Loading dataset: totto

Input: <page_title> List of Spanish regional legislatures </page_title> <section_title> Autonomous Communities </section_title> <table> <cell> Parliament of La Rioja Sp. Parlamento de La Rioja <col_header> Legislature name Local names </col_header> </cell> </table> 

Target: The Parliament of La Rioja (Spanish: Parlamento de La Rioja) is one of the autonomous communities of Spain. 

Ground Truth: ['The Parliament of La Rioja (Spanish: Parlamento de La Rioja) is one of the autonomous communities of Spain.', 'The Parliament of La Rioja (Spanish: Parlamento de La Rioja) is one of the autonomous communities of Spain.', 'The Parliament of La Rioja (Spanish: Parlamento de La Rioja) is one of the autonomous communities of Spain.']


In [4]:
provider = "openai" #ollama, openai, hf, aixplain
add_plan = False # True or False
process_flow = build_agent_workflow(add_plan=add_plan, workers_dict=workers_dict, provider=provider)
# display(Image(process_flow.get_graph(xray=True).draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)))
# display(Image(process_flow.get_graph(xray=True).draw_mermaid_png()))

In [5]:
state = process_flow.invoke(initial_state, config={"recursion_limit": initial_state["recursion_limit"]})
prediction = state['response']

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ORCHESTRATOR OUTPUT:
Thought: The user has provided a request to generate text based on the provided data related to the 'List of Spanish regional legislatures'. The data is shown in a table format, and the relevant content that needs to be sequenced correctly includes the 'Legislature name' and 'Local names'. We need to assign the content ordering worker to properly sequence these pieces of information before moving to the next steps in the pipeline. 

Worker: content ordering
Worker Input: The data provided is as follows:
<page_title> List of Spanish regional legislatures </page_title> 
<section_title> Autonomous Communities </section_title> 
<table> 
<cell> Parliament of La Rioja Sp. Parlamento de La Rioja </cell> 
<col_header> Legislature name Local names </col_header> 
</table>

Your task is to sequence the relevant data fields as they should appear in the final text, focusing on the 'Legislature name' followed by 'Local names'.



> Entering new AgentExecutor chain...


HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```
{
  "action": "Final Answer",
  "action_input": "<page_title> List of Spanish regional legislatures </page_title> <section_title> Autonomous Communities </section_title> <table> <col_header> Legislature name Local names </col_header> <cell> Parliament of La Rioja Sp. Parlamento de La Rioja </cell> </table>"
}
```

> Finished chain.
WORKER: {'input': "The data provided is as follows:\n<page_title> List of Spanish regional legislatures </page_title> \n<section_title> Autonomous Communities </section_title> \n<table> \n<cell> Parliament of La Rioja Sp. Parlamento de La Rioja </cell> \n<col_header> Legislature name Local names </col_header> \n</table>\n\nYour task is to sequence the relevant data fields as they should appear in the final text, focusing on the 'Legislature name' followed by 'Local names'.", 'output': '<page_title> List of Spanish regional legislatures </page_title> <section_title> Autonomous Communities </section_title> <table> <col_header> Legislature name Local names 

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Metric Evaluation Result: 0.358


HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INSPECTOR: The worker's output is correct in terms of content and accurate reflection of the input. However, the metric evaluation result of 0.358 is below the average threshold of 0.6, indicating that the output may not meet the quality expectations required for the task.


HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AGGREGATOR: The Parliament of La Rioja, also known as Parlamento de La Rioja, is part of the Spanish regional legislatures.


In [6]:
save_result_to_json(state, filename=f"{name}_{num}_{add_plan}.json")

Results saved to results/totto_345_False.json


In [7]:
prediction

'The Parliament of La Rioja, also known as Parlamento de La Rioja, is part of the Spanish regional legislatures.'

In [8]:
ground_truth

['The Parliament of La Rioja (Spanish: Parlamento de La Rioja) is one of the autonomous communities of Spain.',
 'The Parliament of La Rioja (Spanish: Parlamento de La Rioja) is one of the autonomous communities of Spain.',
 'The Parliament of La Rioja (Spanish: Parlamento de La Rioja) is one of the autonomous communities of Spain.']

In [9]:
target

'The Parliament of La Rioja (Spanish: Parlamento de La Rioja) is one of the autonomous communities of Spain.'

In [10]:
llm.id

NameError: name 'llm' is not defined

In [ ]:
# from datasets import load_dataset

# rotowire = load_dataset("mrm8488/rotowire-sbnation")
# turku_hockey = datasets.load_dataset('GEM/turku_hockey_data2text')
# totto = datasets.load_dataset('GEM/totto')
# basketball = datasets.load_dataset('GEM/sportsett_basketball')
# webnlg = datasets.load_dataset('GEM/web_nlg', 'en')
# weather = datasets.load_dataset('GEM/conversational_weather')
# dart = datasets.load_dataset('GEM/dart')
# mlb = datasets.load_dataset('GEM/mlb_data_to_text')